## Population projections statistics

Notebook environment to migrate TIF files to CF compliant CoG

In [9]:
# Import standard packages
import os
import glob
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio
#load_dotenv()

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

# use local or remote data dir
use_local_data = False
ds_dirname = "WP5"

if use_local_data: 
    ds_dir = pathlib.Path().home().joinpath("data", ds_dirname)
else: 
    ds_dir = coclico_data_dir.joinpath(ds_dirname, "data", "pop_fp_statistics")

if not ds_dir.exists():
    raise FileNotFoundError("Directory with data does not exist.")


In [ ]:
# List all .csv files from ds_dir
csv_files = list(ds_dir.glob("*.csv"))

[WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_LAU_2010.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_LAU_2030.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_LAU_2050.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_LAU_2100.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_LAU_2150.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_national_2010.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_national_2030.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_national_2050.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_national_2100.csv'),
 WindowsPath('P:/11207608-coclico/FULLTRACK_DATA/WP5/data/pop_fp_statistics/pop_fp_nation

In [41]:
# Open first csv file to check
csv_file = csv_files[0]
df = pd.read_csv(csv_file, sep = ";")

# Print all columns
print(df.columns[8:])

df

Index(['year', 'pop_scenario', 'tot_pop',
       'HIGH_DEFENDED_MAPS_Mean_spring_tide_B01',
       'HIGH_DEFENDED_MAPS_RP_1000_B01', 'HIGH_DEFENDED_MAPS_RP_100_B01',
       'HIGH_DEFENDED_MAPS_RP_1_B01', 'LOW_DEFENDED_MAPS_Mean_spring_tide_B01',
       'LOW_DEFENDED_MAPS_RP_1000_B01', 'LOW_DEFENDED_MAPS_RP_100_B01',
       'LOW_DEFENDED_MAPS_RP_1_B01', 'UNDEFENDED_MAPS_Mean_spring_tide_B01',
       'UNDEFENDED_MAPS_RP_1000_B01', 'UNDEFENDED_MAPS_RP_100_B01',
       'UNDEFENDED_MAPS_RP_1_B01'],
      dtype='object')


,id,Unnamed: 0,LAU_name,GISCO_ID,CNTR_CODE,nuts2,nuts3,LAU_ID,year,pop_scenario,...,HIGH_DEFENDED_MAPS_RP_100_B01,HIGH_DEFENDED_MAPS_RP_1_B01,LOW_DEFENDED_MAPS_Mean_spring_tide_B01,LOW_DEFENDED_MAPS_RP_1000_B01,LOW_DEFENDED_MAPS_RP_100_B01,LOW_DEFENDED_MAPS_RP_1_B01,UNDEFENDED_MAPS_Mean_spring_tide_B01,UNDEFENDED_MAPS_RP_1000_B01,UNDEFENDED_MAPS_RP_100_B01,UNDEFENDED_MAPS_RP_1_B01
0,759,759,Bakar,HR_00043,HR,HR03,HR031,00043,2010,SSP1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000
1,760,760,Bakar,HR_00043,HR,HR03,HR031,00043,2010,SSP1,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,761,761,Bakar,HR_00043,HR,HR03,HR031,00043,2010,SSP1,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,762,762,Bakar,HR_00043,HR,HR03,HR031,00043,2010,SSP1,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,763,763,Bakar,HR_00043,HR,HR03,HR031,00043,2010,SSP1,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59931,46837,6923,Monmouthshire,UK_W06000021,UK,UKL2,UKL21,W06000021,2010,SSP5,...,NaN,NaN,NaN,4.100625,NaN,NaN,NaN,NaN,NaN,NaN
59932,47266,7352,Newport,UK_W06000022,UK,UKL2,UKL21,W06000022,2010,SSP5,...,NaN,NaN,NaN,50.677500,NaN,NaN,NaN,NaN,NaN,NaN
59933,47267,7353,Newport,UK_W06000022,UK,UKL2,UKL21,W06000022,2010,SSP5,...,NaN,NaN,NaN,NaN,NaN,NaN,2381.240625,8672.880625,6955.381250,3738.029375
59934,48545,8631,Powys,UK_W06000023,UK,UKL2,UKL24,W06000023,2010,SSP5,...,NaN,NaN,NaN,NaN,NaN,NaN,0.647500,15.433750,10.250625,2.530625


In [37]:
# Load LAU's from parquet
LAU = gpd.read_parquet(r'p:\11207608-coclico\FULLTRACK_DATA\WP4\LAU_stats\LAU_2020_NUTS_2021_01M_3035_CM.parquet')
LAU

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry
1192,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,DE_03361001,"POLYGON ((4258927.295 3320011.519, 4259193.583..."
1193,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,DE_03361002,"POLYGON ((4264556.862 3317435.819, 4265677.354..."
1195,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,DE_03361003,"POLYGON ((4275479.269 3295636.790, 4275328.930..."
1196,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,DE_03361004,"POLYGON ((4250468.027 3309420.666, 4250432.180..."
1197,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,DE_03361005,"POLYGON ((4272425.673 3310568.711, 4274136.202..."
...,...,...,...,...,...,...,...,...,...,...,...,...
97667,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,FR_97614,"POLYGON ((8727685.023 -2785490.190, 8727139.84..."
97669,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,FR_97616,"POLYGON ((8727114.505 -2787670.745, 8725517.75..."
97670,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,FR_97617,"POLYGON ((8730964.436 -2773602.954, 8730972.75..."
97660,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,FR_97608,"MULTIPOLYGON (((8745852.102 -2769772.861, 8742..."


In [44]:
# Get the columns data of the df in the LAU, match by LAU_ID

# Select only the new columns from the df
df_sel = df.iloc[:, 8:] 
df_sel['LAU_ID'] = df['LAU_ID']  # Add LAU_ID to the df_sel

# Perform a merge on the LAU_ID
LAU_stats = LAU.merge(df_sel, on='LAU_ID', how='inner')

LAU_stats




,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,HIGH_DEFENDED_MAPS_RP_100_B01,HIGH_DEFENDED_MAPS_RP_1_B01,LOW_DEFENDED_MAPS_Mean_spring_tide_B01,LOW_DEFENDED_MAPS_RP_1000_B01,LOW_DEFENDED_MAPS_RP_100_B01,LOW_DEFENDED_MAPS_RP_1_B01,UNDEFENDED_MAPS_Mean_spring_tide_B01,UNDEFENDED_MAPS_RP_1000_B01,UNDEFENDED_MAPS_RP_100_B01,UNDEFENDED_MAPS_RP_1_B01
0,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,0.445,1.335,1.335,1.5575
1,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,NaN,NaN,NaN,1.33500,NaN,NaN,NaN,NaN,NaN,NaN
2,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,NaN,NaN,NaN,NaN,1.33500,NaN,NaN,NaN,NaN,NaN
3,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,NaN,1.55750,NaN,NaN,NaN,1.55750,NaN,NaN,NaN,NaN
4,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,1.33500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61059,SE_0980,SE,SE21,SE214,0980,Gotland,59686.0,18.780720,3178.046351,2020,...,NaN,0.01125,NaN,NaN,NaN,0.01125,NaN,NaN,NaN,NaN
61060,SE_0980,SE,SE21,SE214,0980,Gotland,59686.0,18.780720,3178.046351,2020,...,NaN,NaN,NaN,0.00875,NaN,NaN,NaN,NaN,NaN,NaN
61061,SE_0980,SE,SE21,SE214,0980,Gotland,59686.0,18.780720,3178.046351,2020,...,NaN,NaN,NaN,NaN,0.00875,NaN,NaN,NaN,NaN,NaN
61062,SE_0980,SE,SE21,SE214,0980,Gotland,59686.0,18.780720,3178.046351,2020,...,0.00875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
